In [1]:
# For adding  cities information in cities table in sql database

import pandas as pd
import requests
from datetime import datetime, timedelta
from pytz import timezone
from bs4 import BeautifulSoup

cities = ["Berlin","Mumbai","Hamburg","Pune","Frankfurt","London","Amsterdam","Paris","Munich","Innsbruck","Hannover","New York"]

API_key_city_info = "qClItv2fg4hGDiVLf6zTNw==rLE3xLfQa0uwZFD0"
RAPIAPI_KEY = "687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b"
API_key_weather = '11ada3e0ddad63b0d5c1ac5f93ceb0e9'

#function to create connection string to make connection with Database
def create_connection_string():
  schema = "gans"
  host = "35.241.135.102"
  user = "root"
  password = "Payal122918"
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def retrieve_and_save_cities(cities):
    df_cities_info = create_cities_info(cities)
    save_cities_sql(df_cities_info)
    return "Cities appended to cities table in sql"

def create_cities_info(cities):
    #to save data received in response into a data frame
    name = []
    country = []
    latitude = []
    longitude = []
    city_country_code = []
    for city in cities:
        #https://api.api-ninjas.com/v1/city?name=San Francisco
        api_url = 'https://api.api-ninjas.com/v1/city?name={}'.format(city)
        response = requests.get(api_url, headers={'X-Api-Key': API_key_city_info})
        #if response.status_code == requests.codes.ok:
        #response = requests.get(f"https://api.api-ninjas.com/v1/city?name={city}", headers={'X-Api-Key': API_key_city_info})
        if response.status_code ==200:
            city_info = response.json()
            if len(city_info)>0:
                name.append(city_info[0].get('name'))
                country.append(city_info[0].get('country'))
                latitude.append(city_info[0].get('latitude'))
                longitude.append(city_info[0].get('longitude'))
                #city_country_code.append(f"{city_info[0]['name']} , {city_info[0]['country']}")

    cities_info={'city_name':name,
                'country':country,
                'latitude':latitude,
                'longitude':longitude}
    df_cities_info = pd.DataFrame(cities_info)
    return df_cities_info


def save_cities_sql(df_cities_info):
    #To write cities information to cities table
    connection_string = create_connection_string()
    df_cities_info.to_sql('cities',
                  if_exists='append',
                  con=connection_string,
                  index=False)


retrieve_and_save_cities(cities)

'Cities appended to cities table in sql'

In [5]:
# For adding  populations information in populations table in sql database

import pandas as pd
import requests
from datetime import datetime, timedelta
from pytz import timezone
from bs4 import BeautifulSoup


API_key_city_info = "qClItv2fg4hGDiVLf6zTNw==rLE3xLfQa0uwZFD0"
RAPIAPI_KEY = "687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b"
API_key_weather = '11ada3e0ddad63b0d5c1ac5f93ceb0e9'


#function to create connection string to make connection with Database
def create_connection_string():
  schema = "gans"
  host = "35.241.135.102"
  user = "root"
  password = "Payal122918"
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def fetch_cities_data(connection_string):
  return pd.read_sql("cities", con=connection_string)
    
def create_populations_df():
    connection_string = create_connection_string()
    cities_from_sql = fetch_cities_data(connection_string)
    #cities = cities_from_sql["city_name"]

    population = []
    year_retrieved =[]
    city_id = []
    for index, row in cities_from_sql.iterrows():
        #https://api.api-ninjas.com/v1/city?name=San Francisco
        response = requests.get(f"https://api.api-ninjas.com/v1/city?name={row['city_name']}", headers={'X-Api-Key': API_key_city_info})
        if response.status_code ==200:
            city_info = response.json()
            if len(city_info)>0:
                population.append(city_info[0]['population'])
                city_id.append(row["city_id"])
    
    population_df = pd.DataFrame({"population":population, "city_id": city_id})
    #population_df["city_id"] = cities_from_sql["city_id"]
    population_df["year_retrieved"] = datetime.today().strftime("%Y")
    return population_df


#To write cities information to population table
def retrieve_and_save_populations():
    population_df = create_populations_df()
    save_populations_sql(population_df)
    return "Populations appended to populations table in sql"

def save_populations_sql(population_df):
    connection_string = create_connection_string()
    population_df.to_sql('populations',
                  if_exists='append',
                  con=connection_string,
                  index=False)

retrieve_and_save_populations()

'Populations appended to populations table in sql'

In [7]:
# For adding  airports information in airports table in sql database

import pandas as pd
import requests
from datetime import datetime, timedelta
from pytz import timezone
from bs4 import BeautifulSoup


API_key_city_info = "qClItv2fg4hGDiVLf6zTNw==rLE3xLfQa0uwZFD0"
RAPIAPI_KEY = "687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b"
API_key_weather = '11ada3e0ddad63b0d5c1ac5f93ceb0e9'


#function to create connection string to make connection with Database
def create_connection_string():
  schema = "gans"
  host = "35.241.135.102"
  user = "root"
  password = "Payal122918"
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# To read cities info from cities table
def fetch_cities_data(connection_string):
  return pd.read_sql("cities", con=connection_string)

# to get airports for the cities

    
def icao_airport_codes(cities_df):

  #assert len(latitudes) == len(longitudes)
  latitudes = cities_df['latitude']
  longitudes = cities_df['longitude']
  cities_id = cities_df['city_id']
  list_for_df = []
    
  for index, value in enumerate(latitudes):

    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{value}/{longitudes[index]}/km/50/16"

    querystring = {"withFlightInfoOnly":"true"}

    headers = {
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
      "X-RapidAPI-Key": RAPIAPI_KEY
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    if response.status_code ==200:
        response_df = pd.json_normalize(response.json()['items'])
        response_df['city_id'] = cities_id[index]
        list_for_df.append(response_df)

  
  airports_df = (pd.concat(list_for_df, ignore_index=True))
  ret_airports_df = airports_df[['icao', 'name', 'timeZone', 'city_id']]
  ret_airports_df.rename(columns={"name": "airport_name"}, inplace=True)
  return ret_airports_df


def retrieve_and_save_airports():
    connection_string = create_connection_string()
    ret_cities_df = fetch_cities_data(connection_string)
    airports_df = icao_airport_codes(ret_cities_df)
    airports_df.to_sql('airports',
                  if_exists='append',
                  con=connection_string,
                  index=False)
    return "Airports information retrived and saved in database"


retrieve_and_save_airports()

/var/folders/vs/m818tpl12dx9lgmg3w2kzqch0000gn/T/ipykernel_8573/2174729282.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ret_airports_df.rename(columns={"name": "airport_name"}, inplace=True)


'Airports information retrived and saved in database'

In [3]:
# For adding  weathers information in weathers table in sql database

import pandas as pd
import requests
from datetime import datetime, timedelta
from pytz import timezone
from bs4 import BeautifulSoup


API_key_city_info = "qClItv2fg4hGDiVLf6zTNw==rLE3xLfQa0uwZFD0"
RAPIAPI_KEY = "687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b"
API_key_weather = '11ada3e0ddad63b0d5c1ac5f93ceb0e9'


#function to create connection string to make connection with Database
def create_connection_string():
  schema = "gans"
  host = "35.241.135.102"
  user = "root"
  password = "Payal122918"
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# To read cities info from cities table
def fetch_cities_data(connection_string):
  return pd.read_sql("cities", con=connection_string)

# to get weather data from weather API for cities in the cities 
def fetch_weather_data(cities_df):
  berlin_timezone = timezone('Europe/Berlin')
  weather_items = []

  for _, city in cities_df.iterrows():
      latitude = city["latitude"]
      longitude = city["longitude"]
      city_id = city["city_id"]

      url = (f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={API_key_weather}&units=metric")
      response = requests.get(url)
      if response.status_code ==200:
          weather_data = response.json()

          retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

          for item in weather_data["list"]:
              weather_item = {
                  "city_id": city_id,
                  "forecast_time": item.get("dt_txt"),
                  "temperature": item["main"].get("temp"),
                  "forecast": item["weather"][0].get("main"),
                  "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
                  "snow_in_last_3h": item.get("snow", {}).get("3h", 0),
                  "visibility":item.get("visibility"),
                  "wind_speed": item["wind"].get("speed"),
                  "data_retrieved_at": retrieval_time
              }
              weather_items.append(weather_item)

  weather_df = pd.DataFrame(weather_items)
  weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
  weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"])

  return weather_df

# Function to Save weather data
def save_weather_data(weather_df, connection_string):
  weather_df.to_sql('weathers',
                    if_exists='append',
                    con=connection_string,
                    index=False)


# Read cities data from database
# get weather data from weather api for the cicites in databse
# write weather data retrived above in weather table of database
def retrive_save_weather_info():
    
    connection_string = create_connection_string()
    ret_cities_df = fetch_cities_data(connection_string)
    weather_df = fetch_weather_data(ret_cities_df)
    save_weather_data(weather_df, connection_string)
    return "Weather information retrived and saved in database"


retrive_save_weather_info()

'Weather information retrived and saved in database'

In [9]:
# For adding  flights information in flights table in sql database

import pandas as pd
import requests
from datetime import datetime, timedelta
from pytz import timezone
from bs4 import BeautifulSoup


API_key_city_info = "qClItv2fg4hGDiVLf6zTNw==rLE3xLfQa0uwZFD0"
RAPIAPI_KEY = "687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b"
API_key_weather = '11ada3e0ddad63b0d5c1ac5f93ceb0e9'


#function to create connection string to make connection with Database
def create_connection_string():
  schema = "gans"
  host = "127.0.0.1"
  user = "root"
  password = "rootroot"
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

#to read airports information from sql
def read_airports_data_sql(connection_string):
  return pd.read_sql("airports", con=connection_string)

# to fetch flight information for individual airport with icao = icao_arrival
def get_airport_flights_data(icao_arrival, timezone_str):
    # your code here
    berlin_timezone = timezone(timezone_str)
    today = datetime.now(berlin_timezone).date()
    tomorrow = (today + timedelta(days=1))
    time = ["08:00","20:00"]
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao_arrival}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
    # querystring = {"withLeg":"true",
    #                "direction":"Arrival",
    #                "withCancelled":"false",
    #                "withCodeshared":"true",
    #                "withCargo":"true",
    #                "withPrivate":"true",
    #                "withLocation":"false"}

    querystring = {"withLeg":"true",
                   "direction":"Arrival",
                   "withCancelled":"false",
                   "withCodeshared":"true",
                   "withCargo":"false",
                   "withPrivate":"false"}
    RAPIAPI_KEY = "687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b"
    headers = {
    	"x-rapidapi-key": RAPIAPI_KEY,
    	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code ==200:
        response_json = response.json()
        arrival_items = response_json['arrivals']
        scheduled_arrival_time =[]
        flight_status = [] 
        terminal = []
        flight_number = []
        arrival_airport_icao = []

        retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

        for item in arrival_items:
            scheduled_arrival_time.append(item['arrival']['scheduledTime']['local'])
            flight_status.append(item['status'])
            terminal.append(item['arrival'].get('terminal'))
            flight_number.append(item['number'])
            arrival_airport_icao.append(icao_arrival)

        flights_df = pd.DataFrame({"arrival_airport_icao":arrival_airport_icao,
                                   "flight_number":flight_number,
                                   "scheduled_arrival_time":scheduled_arrival_time,
                                   "flight_status":flight_status,
                                   'terminal':terminal,
                                   "data_retrieved_at": retrieval_time})
        flights_df["scheduled_arrival_time"] = flights_df["scheduled_arrival_time"].str[:-6]
        flights_df["scheduled_arrival_time"] = pd.to_datetime(flights_df["scheduled_arrival_time"])
        flights_df["data_retrieved_at"] = pd.to_datetime(flights_df["data_retrieved_at"])
        return flights_df
    else:
        return 

# to fetch flight data for all airports in the airport table
def fetch_flights_data(airports_data):
    flights_info = []
    for index, row in airports_data.iterrows():
        iceo_arrival = row['icao']
        timezone_str = row['timeZone']
        flights_data = get_airport_flights_data(iceo_arrival,timezone_str)
        flights_info.append(flights_data)

    df_flights_info = pd.concat(flights_info, ignore_index=True)

    return df_flights_info


def retrive_and_save_flights_data():
    connection_string = create_connection_string()
    airports_data = read_airports_data_sql(connection_string)
    df_flights_info = fetch_flights_data(airports_data)
    df_flights_info.to_sql('flights',
                           if_exists='append',
                           con=connection_string,
                           index=False)
    return "Flight information retrived and saved in database"

retrive_and_save_flights_data()

'Flight information retrived and saved in database'